<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/Solar-Fullstack-LLM-101/09_9_RAG_API.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Upstage RAG API (experimental)



## Environment

In [1]:
!pip install -qU langchain-upstage langchain python-dotenv openai rich

### Environment variables

Set up environment variables 
* UPSTAGE_API_KEY


In [2]:
# @title set API key
from pprint import pprint
from rich import print as rprint

import os

import warnings

warnings.filterwarnings("ignore")

if "google.colab" in str(get_ipython()):
    # Running in Google Colab. Please set the UPSTAGE_API_KEY in the Colab Secrets
    from google.colab import userdata

    os.environ["UPSTAGE_API_KEY"] = userdata.get("UPSTAGE_API_KEY")
else:
    # Running locally. Please set the UPSTAGE_API_KEY in the .env file
    from dotenv import load_dotenv

    load_dotenv()

assert (
    "UPSTAGE_API_KEY" in os.environ
), "Please set the UPSTAGE_API_KEY environment variable"

In [3]:
rag_base_url = "https://experimental-api.x.upstage.ai/rag-api/"

# Plase chose your model name
rag_model_name = "my_test_model"

In [4]:
from openai import OpenAI


client = OpenAI(
    base_url=rag_base_url,
    api_key=os.environ["UPSTAGE_API_KEY"]
)

print("Building/Rebuilding the index...")
# Upload all files in the pdfs folder
for file in os.listdir("pdfs"):
    file_path = os.path.join("pdfs", file)
    with open(file_path, "rb") as f:
        file_response = client.files.create(
            file=f,
            purpose="assistants",
            extra_body={"model_name": rag_model_name}
        )
        rprint(file_response)

print("Index build/rebuild submission completed.")

public_model_name = file_response.public_model_name
print("Public Mode Name:", public_model_name)

Building/Rebuilding the index...


FileObject(
    id='file-13',
    bytes=418872,
    created_at=1727456745,
    filename='solar_paper.pdf',
    object='file',
    purpose='assistants',
    status='ready',
    status_details=None,
    model_name='my_test_model',
    public_model_name='788a5ea436f3968df73b',
    message='File solar_paper.pdf is already indexed, skipping indexing'
)

FileObject(
    id='file-14',
    bytes=137425,
    created_at=1727456747,
    filename='solar_sample.pdf',
    object='file',
    purpose='assistants',
    status='ready',
    status_details=None,
    model_name='my_test_model',
    public_model_name='788a5ea436f3968df73b',
    message='File solar_sample.pdf is already indexed, skipping indexing'
)

FileObject(
    id='file-15',
    bytes=878624,
    created_at=1727456748,
    filename='docai.pdf',
    object='file',
    purpose='assistants',
    status='ready',
    status_details=None,
    model_name='my_test_model',
    public_model_name='788a5ea436f3968df73b',
    message='File docai.pdf is already indexed, skipping indexing'
)

FileObject(
    id='file-16',
    bytes=4432693,
    created_at=1727456754,
    filename='kim-tse-2008.pdf',
    object='file',
    purpose='assistants',
    status='error',
    status_details=None,
    model_name='my_test_model',
    public_model_name='788a5ea436f3968df73b',
    message='File kim-tse-2008.pdf is already indexed, skipping indexing'
)

Index build/rebuild submission completed.
Public Mode Name: 788a5ea436f3968df73b


In [5]:
# Import necessary libraries
from pprint import pprint
from langchain_upstage import ChatUpstage as Chat

# Initialize the ChatUpstage model
# Use the previously defined rag_model_name and rag_base_url
chat = Chat(model=rag_model_name, base_url=rag_base_url)

# Send a query to the chat model
question = "What's the Document AI?"
response = chat.invoke(question)

# Print the response content in a formatted way
rprint(response)

AIMessage(
    content='Upstage Document AI is a product that features three solutions: Layout Analyzer, Key Information 
Extractor, and Document OCR.',
    additional_kwargs={'refusal': None},
    response_metadata={
        'token_usage': None,
        'model_name': 'my_test_model',
        'system_fingerprint': None,
        'finish_reason': None,
        'logprobs': None
    },
    id='run-0aaf5a8a-2754-49d5-835a-b1a4884278f9-0'
)

In [6]:
from openai import OpenAI

# Initialize the OpenAI client with custom base URL and API key
client = OpenAI(
    base_url=rag_base_url,  # Using the custom RAG API base URL
    api_key=os.environ["UPSTAGE_API_KEY"]  # Using the API key from environment variables
)

# Send a request to the RAG model
response = client.chat.completions.create(
    model=rag_model_name,  # Using the previously defined RAG model name
    messages=[
        {"role": "user", "content": "Explain Document AI"}  # The user's query
    ],
    stream=False,  # Disable streaming for synchronous response
    extra_body={
        "hybrid_search": True,  # Enable hybrid search for better results
        "contextual_query": True,  # Enable contextual querying
        "verbose": True  # Request verbose output for debugging
    }
)

# Print the model's response
rprint(response)

ChatCompletion(
    id='_',
    choices=[
        Choice(
            finish_reason=None,
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='Upstage Document AI is a product that features three solutions: Layout Analyzer, Key 
Information Extractor, and Document OCR. The Layout Analyzer extracts layouts, tables, and figures from any 
document, while the Key Information Extractor extracts key information from target documents. Document OCR, on the 
other hand, extracts all text from any document. These solutions have been used in various enterprise use cases and
have resulted in higher accuracy (95%-98% accuracy rate) compared to previous manual human work for document 
processing.',
                refusal=None,
                role='assistant',
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1727456764.0,
    model='my_test_model',
    object='chat.completion',
    service_tier=None,
    system_fingerprint=None,
    usage=[
        CompletionUsage(
            completion_tokens=None,
            prompt_tokens=None,
            total_tokens=None,
            completion_tokens_details=None,
            id=None,
            page_content='Document AI: Digitize anythingUpstage Document AI is comprised of three solutions:● 
Layout Analyzer: Extracts layouts, tables, and figures from any document.\n● Key Information Extractor: Extracts 
key information from target documents.\n● Document OCR: Extracts all text from any document.In various enterprise 
use cases, Upstage Document AI resulted in higher accuracy\n(95%-98% accuracy rate) compared to previous manual 
human work for document',
            type='Document'
        ),
        CompletionUsage(
            completion_tokens=None,
            prompt_tokens=None,
            total_tokens=None,
            completion_tokens_details=None,
            id=None,
            page_content='arXiv preprint arXiv:2206.07682.Jason Wei, Xuezhi Wang, Dale Schuurmans, Maarten\nBosma, 
Fei Xia, Ed Chi, Quoc V Le, Denny Zhou,\net al. 2022b. Chain-of-thought prompting elicits rea-\nsoning in large 
language models. Advances in Neural\nInformation Processing Systems, 35:24824–24837.Thomas Wolf, Lysandre Debut, 
Victor Sanh, Julien\nChaumond, Clement Delangue, Anthony Moi, Pier-\nric Cistac, Tim Rault, Rémi Louf, Morgan 
Funtowicz,\net al. 2019. Huggingface’s transformers: State-of-',
            type='Document'
        ),
        CompletionUsage(
            completion_tokens=None,
            prompt_tokens=None,
            total_tokens=None,
            completion_tokens_details=None,
            id=None,
            page_content='the instruction-tuned model is further fine-tuned\nto be more aligned with human or 
strong AI\n(e.g., GPT4 (OpenAI, 2023)) preferences using\nsDPO (Kim et al., 2024a), an improved version\nof direct 
preference optimization (DPO) (Rafailov\net al., 2023). Similar to the instruction tuning stage,\nwe use mostly 
open-source datasets but also syn-\nthesize a math-focused alignment dataset utilizing\nthe ‘Synth. Math-Instruct‘ 
dataset mentioned in the',
            type='Document'
        ),
        CompletionUsage(
            completion_tokens=None,
            prompt_tokens=None,
            total_tokens=None,
            completion_tokens_details=None,
            id=None,
            page_content='(95%-98% accuracy rate) compared to previous manual human work for document\nprocessing. 
Due to its strong performance, the solution has been heavily adopted by various\nleading insurance providers, 
yielding significant productivity gains and operating cost\nreductions.Please find customer testimonials, key 
customers, and customer success stories below.[Key customers]',
            type='Document'
        )
    ]
)

In [7]:
# Initialize the chat model using the public model name (accessible to everyone)
chat = Chat(model=public_model_name, base_url=rag_base_url)

# Define the question about Document AI and its usage
question = "What's Document AI? How can it be used?"

# Invoke the chat model with the question
response = chat.invoke(question)
rprint(response)

AIMessage(
    content='Upstage Document AI is a product that features three solutions: Layout Analyzer, Key Information 
Extractor, and Document OCR. These solutions have been used in various enterprise use cases and have resulted in 
higher accuracy (95%-98% accuracy rate) compared to previous manual human work for document processing.',
    additional_kwargs={'refusal': None},
    response_metadata={
        'token_usage': None,
        'model_name': '788a5ea436f3968df73b',
        'system_fingerprint': None,
        'finish_reason': None,
        'logprobs': None
    },
    id='run-4f859760-a27c-474b-bb02-071801fe6e97-0'
)